In [ ]:
!pip install google-cloud-aiplatform PyPDF2 ratelimit backoff --upgrade --quiet --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
PROJECT_ID = "fluted-card-408220"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import re
import urllib
import warnings
from pathlib import Path

import backoff
import pandas as pd
import PyPDF2
import ratelimit
from google.api_core import exceptions
from tqdm import tqdm
from vertexai.language_models import TextGenerationModel

warnings.filterwarnings("ignore")

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
# Define a folder to store the files
data_folder = "data"
Path(data_folder).mkdir(parents=True, exist_ok=True)

# Define a pdf link to download and place to store the download file
pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = Path(data_folder, pdf_url.split("/")[-1])

# Download the file using `urllib` library
urllib.request.urlretrieve(pdf_url, pdf_file)

(PosixPath('data/practitioners_guide_to_mlops_whitepaper.pdf'),
 <http.client.HTTPMessage at 0x7db2eaf61a80>)

In [ ]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Print three pages from the pdf
for i in range(3):
    text = pages[i].extract_text().strip()

print(f"Page {i}: {text} \n\n")

#text contains only the text from page 2

Page 2: Executive summary
Across industries, DevOps and DataOps have been widely adopted as methodologies to improve quality and re -
duce the time to market of software engineering and data engineering initiatives. With the rapid growth in machine 
learning (ML) systems, similar approaches need to be developed in the context of ML engineering, which handle the 
unique complexities of the practical applications of ML. This is the domain of MLOps. MLOps is a set of standard -
ized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and 
reliably.]
We previously published Google Cloud’s AI Adoption Framework  to provide guidance for technology leaders who 
want to build an effective artificial intelligence (AI) capability in order to transform their business. That framework 
covers AI challenges around people, data, technology, and process, structured in six different themes: learn, lead, 
access, secure, scale, and automate . 
The curre

In [ ]:
# Entry string that contains the extacted text from page 2
print(f"There are {len(text)} characters in the second page of the pdf")

There are 2840 characters in the second page of the pdf


In [ ]:
prompt_template = """
    Write a concise summary of the following text.
    Return your response in bullet points which covers the key points of the text.

    ```{text}```

    BULLET POINT SUMMARY:
"""

In [ ]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=text)

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

- MLOps is a set of standardized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and reliably.
- MLOps framework defines core processes and technical capabilities.
- Organizations can use this framework to help establish mature MLOps practices for building and operationalizing ML systems.
- Adopting the framework can help organizations improve collaboration between teams, improve the reliability and scalability of ML systems, and shorten development cycle times.
- This document is intended for technology leaders and enterprise architects who want to understand MLOps. It’s also for teams who want details about what MLOps looks like in practice. The document assumes that readers are familiar with basic machine learning concepts and with development and deployment practices such as CI/CD.


In [ ]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=text[:30000])

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024)

summary

- MLOps is a set of standardized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and reliably.
- MLOps framework defines core processes and technical capabilities.
- Organizations can use this framework to help establish mature MLOps practices for building and operationalizing ML systems.
- Adopting the framework can help organizations improve collaboration between teams, improve the reliability and scalability of ML systems, and shorten development cycle times.
- This document is intended for technology leaders and enterprise architects who want to understand MLOps. It’s also for teams who want details about what MLOps looks like in practice. The document assumes that readers are familiar with basic machine learning concepts and with development and deployment practices such as CI/CD.